In [ ]:
#import matplotlib
#import matplotlib.pyplot as plt
#plt.style.use(["default"])
#matplotlib.rc('lines',linewidth=3)
#matplotlib.rc('font',family='Nimbus Roman',weight='normal',size=20)
#c_frame=(0,0,0,.8)
#for tick in 'xtick','ytick':
#    matplotlib.rc(tick+'.major',width=1.5,size=8)
#    matplotlib.rc(tick+'.minor',width=1,size=4,visible=True)
#    matplotlib.rc(tick,color=c_frame,labelsize=18,direction='in')
#matplotlib.rc('xtick',top=True)
#matplotlib.rc('ytick',right=True)
#matplotlib.rc('axes',linewidth=1.5,edgecolor=c_frame,labelweight='normal')
#matplotlib.rc('grid',color=c_frame)
#matplotlib.rc('patch',edgecolor=c_frame)
#from matplotlib import rcParams
#rcParams['mathtext.fontset']='custom'
#rcParams['mathtext.rm']='Nimbus Roman'
#rcParams['mathtext.it']='Nimbus Roman'
#rcParams['mathtext.bf']='Nimbus Roman'
from pyhipp import plot
plot.runtime_config.use_stylesheets('mathtext-it')

In [ ]:
import sys
#sys.path.append("/home/cyma")
#sys.path.append("/home/cyma/illustris_python")
import illustris_python as il
import illustris_python.snapshot
import illustris_python.groupcat
#import illustris_python.add_func
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from matplotlib.ticker import AutoMinorLocator
from astropy.io import fits
import statistics
from pathlib import Path
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

class ColorSets:
    dark2 = plot.ColorSeq.predefined('dark2').get_rgba()
    tab10 = plot.ColorSeq.predefined('tab10').get_rgba()
    set1 = plot.ColorSeq.predefined('set1').get_rgba()
    set2 = plot.ColorSeq.predefined('set2').get_rgba()
    
    def __getitem__(self, name):
        if isinstance(name, tuple):
            return tuple(self[n] for n in name) 
        return getattr(self, name)

c_sets = ColorSets()

class NamedColors:
    k               = 'black'
    r               = c_sets.set1[0] 
    red             = c_sets.set1[0]
    pink            = '#c284b3'
    b               = c_sets.set1[1] 
    blue            = c_sets.set1[1]
    p               = c_sets.dark2[2] 
    purple          = c_sets.dark2[2]
    ly              = c_sets.dark2[5] 
    lightyellow     = c_sets.dark2[5]
    y               = c_sets.dark2[6] 
    yellow          = c_sets.dark2[6]
    g               = c_sets.dark2[0] 
    green           = c_sets.dark2[0]
    lg              = c_sets.set1[2] 
    lightgreen      = c_sets.set1[2]
    grey            = c_sets.dark2[-1]
    orange          = c_sets.dark2[1] 
    lightorange     = c_sets.set1[4]
    
    def __getitem__(self, name):
        if isinstance(name, tuple):
            return tuple(self[n] for n in name) 
        return getattr(self, name)
    

cs_named = NamedColors()

plot_data_dir = Path("./data")
out_fig_dir = Path("./figures")

panel_number_kw = dict(fontfamily='Arial', fontdict={'weight':'bold'}, fontsize=17)

### Plotting

In [ ]:
hdul=fits.open(plot_data_dir/"Nunki0010mis.fits")
data=hdul[1].data
hdul.close()
w6module=data['w6module']
w6moduleperc1=data['w6moduleperc1']
w6moduleperc2=data['w6moduleperc2']
w6=data['w6']
w6perc1=data['w6perc1']
w6perc2=data['w6perc2']
w6cos=data['w6cos']
w6cosperc1=data['w6cosperc1']
w6cosperc2=data['w6cosperc2']

In [ ]:
part1=np.arange(0,40,1)
part2=np.arange(40,200,5)
part3=np.arange(200,400,20)
part4=np.arange(400,2001,100)
result=np.concatenate([part1,part2,part3,part4])
ybinsabs=np.arange(1,-0.01,-0.04)
xbins=result
print(result,len(result))

In [ ]:
fig, axs = plot.subplots((2,1), share=(True,False), 
                         space=0.025, 
                         subsize=(6.0, 3.25), 
                         margin=[0.025, 0.025, 0.085, 0.15], layout='none')
axs_f = axs.flat

ax = axs_f[0]
ax.plot(xbins[1:],w6module, lw=3.,label=r'$\left|\vec{\lambda}\right|$')
ax.fill_between(xbins[1:],w6moduleperc1,w6moduleperc2,alpha=0.5,
                color='grey')
ax.plot(xbins[1:],w6, lw=3., c=cs_named['b'], label=r'$\lambda_z$')
ax.fill_between(xbins[1:],w6perc1,w6perc2,alpha=0.3,color=cs_named['b'])

ax = axs_f[1]
ax.plot(xbins[1:],w6cos,lw=3)
ax.fill_between(xbins[1:],w6cosperc1,w6cosperc2,alpha=0.5,
                color='grey')

axs_f[0].label(y=r'\text{Spin}')\
    .scale('log','log')\
    .lim([1., 2.0e3], [10**-3.05, 10**1.25])\
    .leg(loc='ul', handlelength=1.2, labelcolor='linecolor')
axs_f[1].label(r'r\,[\text{kpc}]',r'\cos\,\left<\vec{\lambda},z\right>')\
    .lim(y=[-0.65, 1.25])
pan_labs = 'a', 'b'    
for i_ax, ax in enumerate(axs_f):
    ax.text(r'\,$%s$\,'%(pan_labs[i_ax]), (.94, .9), **panel_number_kw, ha='center')
plot.savefig(out_fig_dir/'spinalignment.pdf')

In [ ]:
fig=plt.figure(figsize=(8,8))
ax1=fig.add_subplot(211)
ax1.set_ylabel('$\\lambda$')
ax1.set_yscale('log')
ax1.plot(xbins[1:],w6module,'lime')
ax1.plot(xbins[1:],w6,'r')
ax1.fill_between(xbins[1:],w6moduleperc1,w6moduleperc2,alpha=0.3,color='lime')
ax1.fill_between(xbins[1:],w6perc1,w6perc2,alpha=0.3,color='r')
ax1.text(0.1,0.1,'a',ha='center',va='center',fontsize=36,color='black',fontweight='bold',transform=ax1.transAxes)
ax2=fig.add_subplot(212,sharex=ax1)
ax2.set_xlabel('r/kpc')
ax2.set_ylabel('cos $\\theta$')
ax2.set_xscale('log')
ax2.plot(xbins[1:],w6cos,'r')
ax2.fill_between(xbins[1:],w6cosperc1,w6cosperc2,alpha=0.3,color='r')
ax2.text(0.1,0.1,'b',ha='center',va='center',fontsize=36,color='black',fontweight='bold',transform=ax2.transAxes)
ax2.set_xlim([1,2000])
plt.setp(ax1.get_xticklabels(),visible=False)
plt.subplots_adjust(hspace=0,left=0.12,right=0.98,top=0.98,bottom=0.08)
# fig.savefig('/home/cyma/Sagittarius/spinalignment.pdf')

### Computation

In [ ]:
basePath='/data/public/TNG50-1/output/'
snapshot=99
a=1
fieldsA=['SubhaloFlag','SubhaloMassInRadType','SubhaloHalfmassRadType','SubhaloGrNr','SubhaloSFRinRad','SubhaloVel','SubhaloPos','SubhaloSpin']
subhalos=il.groupcat.loadSubhalos(basePath,snapshot,fieldsA)
x0=subhalos['SubhaloFlag']
x1=subhalos['SubhaloMassInRadType'][:,4]/0.6774
x2=subhalos['SubhaloHalfmassRadType'][:,4]/0.6774*a
x3=subhalos['SubhaloGrNr']
x4=subhalos['SubhaloSFRinRad']
x6=subhalos['SubhaloVel']
x7=subhalos['SubhaloPos']/0.6774*a
x8=subhalos['SubhaloSpin']
hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/galid.fits")
data=hdul[1].data
hdul.close()
idxA=data["galid"]
x1a=x1[idxA]
x2a=x2[idxA]
x3a=x3[idxA]
x4a=x4[idxA]
x6a=x6[idxA]
x7a=x7[idxA]
x8a=x8[idxA]
print(len(idxA))

In [ ]:
GroupFirstSub=il.groupcat.loadHalos(basePath,snapshot,'GroupFirstSub')[x3]
Group_R_Crit200=il.groupcat.loadHalos(basePath,snapshot,'Group_R_Crit200')[x3]/0.6774*a
Group_M_Crit200=il.groupcat.loadHalos(basePath,snapshot,'Group_M_Crit200')[x3]/0.6774
Group_R_Crit200a=Group_R_Crit200[idxA]
Group_M_Crit200a=Group_M_Crit200[idxA]
vr=np.sqrt(Group_M_Crit200a*Group_R_Crit200a)*207.41
print(np.nanmedian(x2a),np.nanmedian(Group_R_Crit200a))

In [ ]:
x1bh=subhalos['SubhaloMassInRadType'][:,5]/0.6774
x1bha=x1bh[idxA]

In [ ]:
idxAb=np.where((np.log10(x4a)>0.74878283*(np.log10(x1a)+10)-8.7082215)&(np.log10(x1bha)<-1.8))[0]
print(idxAb)
print(len(idxAb),np.nanmedian(Group_R_Crit200a[idxAb]))

In [ ]:
hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/galplane.fits")
data=hdul[1].data
hdul.close()
plane=data["plane"]

In [ ]:
part1=np.arange(0,40,1)
part2=np.arange(40,200,5)
part3=np.arange(200,400,20)
part4=np.arange(400,2001,100)
result=np.concatenate([part1,part2,part3,part4])
ybinsabs=np.arange(1,-0.01,-0.04)
print(result,len(result))

In [ ]:
xbins=result
wVi=np.zeros((len(idxA),len(xbins)-1))
wvrr2rhoi=np.zeros((len(idxA),len(xbins)-1))
wVo=np.zeros((len(idxA),len(xbins)-1))
wvrr2rhoo=np.zeros((len(idxA),len(xbins)-1))
wm=np.zeros((len(idxA),len(xbins)-1))
wV=np.zeros((len(idxA),len(xbins)-1))
wlax=np.zeros((len(idxA),len(xbins)-1))
wlay=np.zeros((len(idxA),len(xbins)-1))
wlaz=np.zeros((len(idxA),len(xbins)-1))
wvrr2rho=np.zeros((len(idxA),len(xbins)-1))
for k in range(len(idxA)):
    spin=x8a[k]
    coordinate=x7a[k]
    velocity=x6a[k]
    hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/Alnilam{k}.fits")
    data=hdul[1].data
    hdul.close()
    coordinategasB=data["coordinategas"]
    hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/Alnitak{k}.fits")
    data=hdul[1].data
    hdul.close()
    velocitygasB=data["velocitygas"]
    hdul=fits.open(f"/data/public/TNG50-1/cymadeposit/idx0010/Betelgeuse{k}.fits")
    data=hdul[1].data
    hdul.close()
    massgasB=data["massgas"]
    dengasB=data["densitygas"]
    print(k)
    gamar=plane[k]
    coordinategasB=il.add_func.func_face_onnew(coordinategasB,gamar,coordinate)
    velocitygasB=il.add_func.func_face_onnew(velocitygasB,gamar,velocity)+coordinategasB*0.06774
    rgasB=np.sqrt(np.sum(coordinategasB*coordinategasB,axis=1))
    vrgasB=np.sum(velocitygasB*coordinategasB,axis=1)/rgasB
    lax=velocitygasB[:,2]*coordinategasB[:,1]-velocitygasB[:,1]*coordinategasB[:,2]
    lay=velocitygasB[:,0]*coordinategasB[:,2]-velocitygasB[:,2]*coordinategasB[:,0]
    laz=velocitygasB[:,1]*coordinategasB[:,0]-velocitygasB[:,0]*coordinategasB[:,1]
    print(k)
    idxC=((rgasB<2000)&(vrgasB<0))
    [wVi[k],
     wvrr2rhoi[k]]=il.add_func.multi_histogram1d(rgasB[idxC],
                                                 weights_list=[massgasB[idxC]/dengasB[idxC],
                                                               massgasB[idxC]*vrgasB[idxC]*rgasB[idxC]*rgasB[idxC]],
                                                 bins=xbins)
    idxC=((rgasB<2000)&(vrgasB>0))
    [wVo[k],
     wvrr2rhoo[k]]=il.add_func.multi_histogram1d(rgasB[idxC],
                                                 weights_list=[massgasB[idxC]/dengasB[idxC],
                                                               massgasB[idxC]*vrgasB[idxC]*rgasB[idxC]*rgasB[idxC]],
                                                 bins=xbins)
    idxC=(rgasB<2000)
    [wm[k],
     wV[k],
     wlax[k],
     wlay[k],
     wlaz[k],
     wvrr2rho[k]]=il.add_func.multi_histogram1d(rgasB[idxC],
                                                weights_list=[massgasB[idxC],
                                                              massgasB[idxC]/dengasB[idxC],
                                                              massgasB[idxC]*lax[idxC],
                                                              massgasB[idxC]*lay[idxC],
                                                              massgasB[idxC]*laz[idxC],
                                                              massgasB[idxC]*vrgasB[idxC]*rgasB[idxC]*rgasB[idxC]],
                                                bins=xbins)
    print(k)

In [ ]:
w7b=wvrr2rho[idxAb]/wV[idxAb]
w7ib=wvrr2rhoi[idxAb]/wVi[idxAb]
w7ob=wvrr2rhoo[idxAb]/wVo[idxAb]
w6xb=wlax[idxAb]/wm[idxAb]/np.tile(vr[idxAb,np.newaxis],(1,len(xbins)-1))
w6yb=wlay[idxAb]/wm[idxAb]/np.tile(vr[idxAb,np.newaxis],(1,len(xbins)-1))
w6zb=wlaz[idxAb]/wm[idxAb]/np.tile(vr[idxAb,np.newaxis],(1,len(xbins)-1))
w6moduleb=np.sqrt(w6xb*w6xb+w6yb*w6yb+w6zb*w6zb)
w6cosb=w6zb/w6moduleb

In [ ]:
w7=np.nanmedian(w7b,axis=0).T*10.23
w7i=np.nanmedian(w7ib,axis=0).T*10.23
w7o=np.nanmedian(w7ob,axis=0).T*10.23
w6=np.nanmedian(w6zb,axis=0).T/np.sqrt(2)
w6module=np.nanmedian(w6moduleb,axis=0).T/np.sqrt(2)
w6cos=np.nanmedian(w6cosb,axis=0).T

In [ ]:
perc1=16
perc2=84
w6perc1=np.nanpercentile(w6zb,perc1,axis=0).T/np.sqrt(2)
w6moduleperc1=np.nanpercentile(w6moduleb,perc1,axis=0).T/np.sqrt(2)
w6cosperc1=np.nanpercentile(w6cosb,perc1,axis=0).T
w6perc2=np.nanpercentile(w6zb,perc2,axis=0).T/np.sqrt(2)
w6moduleperc2=np.nanpercentile(w6moduleb,perc2,axis=0).T/np.sqrt(2)
w6cosperc2=np.nanpercentile(w6cosb,perc2,axis=0).T

In [ ]:
fig=plt.figure(figsize=(4,4))
ax=fig.add_subplot(111)
ax.set_xlim([1,2000])
ax.set_xscale('log')
ax.set_xlabel('r/kpc')
ax.set_ylabel('$\\Phi$ [M$_\\odot$/yr]')
ax.set_title('10$^{10}$M$_\\odot$ z=0')
ax.plot(xbins[1:],w7*4*np.pi,'g')
ax.plot(xbins[1:],w7i*4*np.pi,'b')
ax.plot(xbins[1:],w7o*4*np.pi,'r')